<a href="https://colab.research.google.com/github/sheldonkemper/portfolio/blob/main/CAM_DS_LangChain_Advanced_Concepts_2_1_1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**First things first** - please go to 'File' and select 'Save a copy in Drive' so that you have your own version of this activity set up and ready to use.
Remember to update the portfolio index link to your own work once completed!

#Demonstration 2.1.1.b LangChain advanced concepts

In this demonstration, you will load an LLM and install LangChain to enable several useful applications. You will learn how to:
- Structure the output in a defined format.
- Create chains that connect prompt outputs to new inputs.
- Give your LLM a memory for conversational interaction.


**Important**: The demonstration uses closed-source models from OpenAI that require API keys. You will be advised to register for an account at the OpenAI developer platform if you do not already have one. The provision of API keys is restricted to personal usage only and is subject to OpenAI’s rate limits. At the time of writing this programme, a sufficient quota of API keys was being offered without charge, but a recent change at OpenAI required that anyone requesting free keys had to add a small credit to their account for the query to work. You will be reimbursed for this credit.

#### Get your OpenAI key

1. Log in at [OpenAI developer platform](https://platform.openai.com/api-keys).
2. Create a new secret key.
3. Copy and paste the key into a document for safe-keeping.
4. Paste the key below, where it says 'Replace with API key'.



In [ ]:
!pip install python-dotenv
!pip install openai
!pip install  torch transformers accelerate bitsandbytes transformers sentence-transformers
!pip install --upgrade langchain

In [ ]:
!pip install --upgrade langchain

In [ ]:
!pip install -U langchain-community

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = 'REPLACE WITH YOUR API KEY'


### Output parsers

In [ ]:
customer_feedback = """
My name is Paul, I recently stayed at the Blue Horizon Hotel, \
and while the view was stunning, the room service was disappointing. \
I ordered breakfast at 9 AM, but it didn't arrive until 11 AM. \
The staff was friendly but seemed overwhelmed, and the room was not as clean as I'd expected.
The Wi-Fi was slow, and I had to use my phone's hotspot to get any work done. However, the location was great, and I enjoyed the pool. """


feedback_template = """
For the following information, extract the following information
name: the name of the person
venue: the venue where the customer stayed
comment: summary of the feebdack by the customer
service: service the custome booked

Format the output as JSON with the following keys:
name
venue
comment
service

Text: {text}

"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(feedback_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nFor the following information, extract the following information\nname: the name of the person\nvenue: the venue where the customer stayed\ncomment: summary of the feebdack by the customer\nservice: service the custome booked\n\nFormat the output as JSON with the following keys:\nname\nvenue\ncomment\nservice\n\nText: {text}\n\n'), additional_kwargs={})]


In [ ]:
messages = prompt_template.format_messages(text=customer_feedback )

In [ ]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(model_name="gpt-3.5-turbo")
response = chat(messages)
print(response.content)

{
  "name": "Paul",
  "venue": "Blue Horizon Hotel",
  "comment": "While the view was stunning, the room service was disappointing. The staff was friendly but seemed overwhelmed, and the room was not as clean as I'd expected. The Wi-Fi was slow. However, the location was great, and I enjoyed the pool.",
  "service": "Room service"
}


In [ ]:
type(response.content)

str

In [ ]:
# Note that you will get an error by running this line of code
# because 'name' is not a dictionary;
# 'name' is a string.
response.content.get('name')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate,PromptTemplate
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [ ]:
customer_feedback = """
My name is Paul, I recently stayed at the Blue Horizon Hotel, \
and while the view was stunning, the room service was disappointing. \
I ordered breakfast at 9 AM, but it didn't arrive until 11 AM. \
The staff was friendly but seemed overwhelmed, and the room was not as clean as I'd expected.
The Wi-Fi was slow, and I had to use my phone's hotspot to get any work done. However, the location was great, and I enjoyed the pool. """


feedback_template = """
For the following information, extract the following information
name: the name of the person
venue: the venue where the customer stayed
comment: summary of the feebdack by the customer
service: service the custome booked

Format the output as JSON with the following keys:
name
venue
comment
service

Text: {text}

"""

In [ ]:
# Consider how you would like your response structured. This is essentially an advanced prompt template.
response_schemas = [
    ResponseSchema(name="name", description="the name of the person"),
    ResponseSchema(name="venue", description="the venue where the customer stayed"),
    ResponseSchema(name="comment", description="summary of the feebdack by the customer"),
    ResponseSchema(name="service", description=" service the custome booked"),
]

# Conside how you would like to parse your output.
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
# See the prompt template you created for formatting.
format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"name": string  // the name of the person
	"venue": string  // the venue where the customer stayed
	"comment": string  // summary of the feebdack by the customer
	"service": string  //  service the custome booked
}
```


In [ ]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input=customer_feedback )

print(promptValue)


You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"name": string  // the name of the person
	"venue": string  // the venue where the customer stayed
	"comment": string  // summary of the feebdack by the customer
	"service": string  //  service the custome booked
}
```

% USER INPUT:

My name is Paul, I recently stayed at the Blue Horizon Hotel, and while the view was stunning, the room service was disappointing. I ordered breakfast at 9 AM, but it didn't arrive until 11 AM. The staff was friendly but seemed overwhelmed, and the room was not as clean as I'd expected.
The Wi-Fi was slow, and I had to use my phone's hotspot to get any work done. However, the location was great, and I enjoyed the pool. 

YOUR RESPONSE:



In [ ]:
print(messages[0].content)


For the following information, extract the following information
name: the name of the person
venue: the venue where the customer stayed
comment: summary of the feebdack by the customer
service: service the custome booked

Format the output as JSON with the following keys:
name
venue
comment
service

Text: 
My name is Paul, I recently stayed at the Blue Horizon Hotel, and while the view was stunning, the room service was disappointing. I ordered breakfast at 9 AM, but it didn't arrive until 11 AM. The staff was friendly but seemed overwhelmed, and the room was not as clean as I'd expected.
The Wi-Fi was slow, and I had to use my phone's hotspot to get any work done. However, the location was great, and I enjoyed the pool. 




In [ ]:
llm_output = llm(promptValue)
llm_output

'\n```json\n{\n\t"name": "Paul",\n\t"venue": "Blue Horizon Hotel",\n\t"comment": "The view was stunning, but the room service was disappointing. Ordered breakfast at 9 AM, but it didn\'t arrive until 11 AM. The staff was friendly but seemed overwhelmed, and the room was not as clean as expected. The Wi-Fi was slow, and had to use phone\'s hotspot to get work done. However, the location was great, and enjoyed the pool.",\n\t"service": "Room service"\n}\n```'

In [ ]:
output_dict = output_parser.parse(llm_output)

In [ ]:
output_dict

{'name': 'Paul',
 'venue': 'Blue Horizon Hotel',
 'comment': "The view was stunning, but the room service was disappointing. Ordered breakfast at 9 AM, but it didn't arrive until 11 AM. The staff was friendly but seemed overwhelmed, and the room was not as clean as expected. The Wi-Fi was slow, and had to use phone's hotspot to get work done. However, the location was great, and enjoyed the pool.",
 'service': 'Room service'}

In [ ]:
output_dict['name']

'Paul'

## Chains

In [ ]:
from langchain.chains import LLMChain, SimpleSequentialChain


# Create the first prompt template.
template1 = PromptTemplate(
    input_variables=["topic"],
    template="Write a title for a research paper on the topic: '{topic}'"
)

# Create the first LLM chain.
chain1 = LLMChain(llm=llm, prompt=template1)

# Create the second prompt template.
template2 = PromptTemplate(
    input_variables=["title"],
    template="Write an abstract for a research paper with the title: '{title}'"
)

# Create the second LLM chain.
chain2 = LLMChain(llm=llm, prompt=template2)



In [ ]:
# Combine the two chains into a SimpleSequentialChain.
sequential_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose = True)

# Provide the input topic.
input_topic = "Applications of Artificial Intelligence in Healthcare"

# Run the chain.
result = sequential_chain.invoke(input_topic)

# Output the result.
print(result)




> Entering new SimpleSequentialChain chain...


"Revolutionizing Healthcare: Exploring the Transformative Impact of Artificial Intelligence Applications"


This research paper explores the transformative impact of artificial intelligence (AI) applications in the healthcare industry. With the increasing demand for efficient and effective healthcare services, AI has emerged as a promising solution. The paper delves into the various ways in which AI is revolutionizing the healthcare landscape, including its potential to improve patient outcomes, reduce costs, and enhance overall healthcare delivery. Drawing on a review of existing literature and case studies, the paper highlights the current state of AI applications in healthcare and identifies key challenges and opportunities for its future implementation. Furthermore, the paper discusses ethical and privacy concerns related to the use of AI in healthcare and proposes potential solutions to address them. Through this research, it is ho

## Conversation  memory

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.llms import OpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo")

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [ ]:
template_string =  """
You are an AI assistant that gives answers to the user questions
Question: {query}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [ ]:
prompt = prompt_template.format_messages(
  query = "Hi, My name is Paul")

response = chat(prompt)
print(response.content)


Answer: Hello Paul! How can I assist you today?


In [ ]:
prompt = prompt_template.format_messages(
  query = "What is data science")

response = chat(prompt)
print(response.content)


Data science is a multidisciplinary field that uses scientific methods, algorithms, and systems to extract insights and knowledge from structured and unstructured data. It combines principles from statistics, computer science, and domain-specific knowledge to analyze and interpret complex data sets. Data scientists use various techniques such as data mining, machine learning, and data visualization to uncover patterns, trends, and insights that can be used to make informed decisions and predictions.


In [ ]:
prompt = prompt_template.format_messages(
  query = "What did I say my name was?")

response = chat(prompt)
print(response.content)

I'm sorry, I do not have the capability to remember personal information such as your name. Please feel free to remind me what you said your name was.


### Conversation chain

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=chat,
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="Hi, My name is Paul")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, My name is Paul
AI:

> Finished chain.


"Hello Paul! It's nice to meet you. How can I assist you today?"

In [ ]:
conversation.predict(input="What is data science")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, My name is Paul
AI: Hello Paul! It's nice to meet you. How can I assist you today?
Human: What is data science
AI:

> Finished chain.


'Data science is a multidisciplinary field that uses scientific methods, processes, algorithms, and systems to extract knowledge and insights from structured and unstructured data. It combines principles from statistics, computer science, and domain-specific expertise to analyze and interpret complex data sets. Data science is used in various industries such as healthcare, finance, marketing, and more to make data-driven decisions and predictions. Is there anything specific you would like to know about data science, Paul?'

In [ ]:
conversation.predict(input="What did I say my name was?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, My name is Paul
AI: Hello Paul! It's nice to meet you. How can I assist you today?
Human: What is data science
AI: Data science is a multidisciplinary field that uses scientific methods, processes, algorithms, and systems to extract knowledge and insights from structured and unstructured data. It combines principles from statistics, computer science, and domain-specific expertise to analyze and interpret complex data sets. Data science is used in various industries such as healthcare, finance, marketing, and more to make data-driven decisions and predictions. Is there anything specific you would like to know about data science, Paul?
Human: What did I say my

'You said your name is Paul.'

In [ ]:
print(memory.buffer)

Human: Hi, My name is Paul
AI: Hello Paul! It's nice to meet you. How can I assist you today?
Human: What is data science
AI: Data science is a multidisciplinary field that uses scientific methods, processes, algorithms, and systems to extract knowledge and insights from structured and unstructured data. It combines principles from statistics, computer science, and domain-specific expertise to analyze and interpret complex data sets. Data science is used in various industries such as healthcare, finance, marketing, and more to make data-driven decisions and predictions. Is there anything specific you would like to know about data science, Paul?
Human: What did I say my name was?
AI: You said your name is Paul.


### Conversation buffer window memory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory = ConversationBufferWindowMemory(k=1)

In [ ]:

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

In [ ]:
conversation.predict(input="Hi, My name is Paul")

" Hello Paul! My name is AI, which stands for Artificial Intelligence. It's nice to meet you. What can I assist you with today?"

In [ ]:
conversation.predict(input="What is data science")

' Data science is a multidisciplinary field that combines statistics, computer science, and domain knowledge to extract insights and knowledge from data. It involves collecting, cleaning, and analyzing large sets of data to identify patterns and make data-driven decisions. Some common techniques used in data science include machine learning, data mining, and data visualization. Does that answer your question?'

In [ ]:
conversation.predict(input="What did I say my name was?")

' I do not have access to your personal information, so I do not know what name you said. Would you like to tell me?'

In [ ]:
memory = ConversationBufferWindowMemory(k=5)
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

In [ ]:
conversation.predict(input="Hi, My name is Paul")

" Hello Paul! It's nice to meet you. My name is AI. I am an artificial intelligence designed to assist and communicate with humans. How are you doing today?"

In [ ]:
conversation.predict(input="What is data science")

' Data science is a multidisciplinary field that combines statistics, computer science, and domain expertise to extract valuable insights and knowledge from large and complex datasets. It involves collecting, processing, analyzing, and interpreting data to solve real-world problems and make informed decisions. Data scientists use various tools and techniques such as programming languages, machine learning algorithms, and data visualization to uncover patterns and trends in data. It is a rapidly growing field with applications in various industries such as healthcare, finance, marketing, and more. Is there anything else you would like to know about data science?'

In [ ]:
conversation.predict(input="What did I say my name was?")

' You said your name is Paul. Is there any other information you would like to know?'

## Key information
You have learned how to perform several advanced applications of LangChain, including output parsing, chaining prompts, and preserving conversation memory.

## Reflect
Besides the examples given, can you think of a real-life scenario for each of these applications? Note your ideas in the space below.

> Select the pen from the toolbar to add your entry.